In [88]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import datetime
import requests
from bs4 import BeautifulSoup as bs
import json

In [89]:
#----------資料讀入並進行篩選----------
CrudeOilETF= pd.read_csv('Crude Oil ETF List (22).csv') #讀檔
fliter = (CrudeOilETF["Inception"] < '2015-12-31') #Inception條件判斷進行篩選
newCrudeOilETF = CrudeOilETF[fliter] #Crude Oil ETF List篩選結果

GoldETF= pd.read_csv('Gold ETF List (17).csv') #讀檔
fliter = (GoldETF["Inception"] < '2015-12-31') #Inception條件判斷進行篩選
newGoldETF = GoldETF[fliter] #Gold ETF List篩選結果

#將符合的ETF代碼轉換為List
CrudeOilETFList = newCrudeOilETF['Symbol'].values.tolist()
GoldETList = newGoldETF['Symbol'].values.tolist() 

SymbolList = CrudeOilETFList+GoldETList
print(SymbolList)

['USO', 'UCO', 'DBO', 'SCO', 'USL', 'UWTI', 'DTO', 'DWTI', 'OLO', 'OLEM', 'SZO', 'GLD', 'IAU', 'SGOL', 'OUNZ', 'UGLD', 'DGL', 'UGL', 'DGP', 'GLL', 'DZZ', 'DGLD', 'DGZ', 'UBG']


In [90]:
def crawler(ETF,startDate,endDate,freq,target):
    dict={}
    date_arr=[]
    df = pd.DataFrame()
    page_url="https://query1.finance.yahoo.com/v8/finance/chart/USO?symbol="+ETF+"&period1="+str(startDate)+"&period2="+str(endDate)+"&interval="+freq+"&includePrePost=true&events=div%7Csplit%7Cearn&lang=en-US&region=US&crumb=Tgw6xSDlmOe&corsDomain=finance.yahoo.com"
    r = requests.get(page_url)
    soup = BeautifulSoup(r.text, 'lxml')
    a=soup.p.get_text()
    python_dict = json.loads(a)
    if (python_dict['chart']['result'] == None):
        print("已下市")
    else:
        timestamp=python_dict['chart']['result'][0]['timestamp']
        for time in timestamp:
            date_arr.append(datetime.datetime.fromtimestamp(time).strftime("%Y-%m-%d"))
            dict['Date']=date_arr
        dict[ETF]=python_dict['chart']['result'][0]['indicators'][target][0][target]
        if(freq == '1wk' or freq == '1mo'):
            dict['Date'].pop() #移除最後一個
            dict[ETF].pop() #移除最後一個
        df = pd.DataFrame.from_dict(dict)
        df.set_index("Date" , inplace=True)
    return df

In [91]:
now=datetime.datetime.now()
import time
now = int(time.mktime(now.timetuple()))
start_time=int(time.mktime(datetime.date(2016, 1, 1).timetuple()))



def get_Date(freq,target,etfs):
    print('----------'+freq+'----------')
    all_data = pd.DataFrame()
    ETF_syb = []
    for Symbol in etfs:
        print('start crawling :', Symbol)
        data = crawler(Symbol,start_time,now,freq,target)
        datalen = data.shape[0]
        if datalen == 0:
            continue
        ETF_syb.append(Symbol)
        if datalen > all_data.shape[0]:
            all_data = pd.concat([all_data, data], axis=1,join_axes=[data.index])
        else:
            all_data = pd.concat([all_data, data], axis=1,join_axes=[all_data.index])
    return all_data,ETF_syb

In [93]:
all_data_week,ETFLISTwk = get_Date('1wk','adjclose',SymbolList)
all_data_month,ETFLISTmo = get_Date('1mo','adjclose',SymbolList)
print(ETFLISTwk)
print(ETFLISTmo)

----------1wk----------
start crawling : USO
start crawling : UCO
start crawling : DBO
start crawling : SCO
start crawling : USL
start crawling : UWTI
已下市
start crawling : DTO
start crawling : DWTI
已下市
start crawling : OLO
start crawling : OLEM
start crawling : SZO
start crawling : GLD
start crawling : IAU
start crawling : SGOL
start crawling : OUNZ
start crawling : UGLD
start crawling : DGL
start crawling : UGL
start crawling : DGP
start crawling : GLL
start crawling : DZZ
start crawling : DGLD
start crawling : DGZ
start crawling : UBG
----------1mo----------
start crawling : USO
start crawling : UCO
start crawling : DBO
start crawling : SCO
start crawling : USL
start crawling : UWTI
已下市
start crawling : DTO
start crawling : DWTI
已下市
start crawling : OLO
start crawling : OLEM
start crawling : SZO
start crawling : GLD
start crawling : IAU
start crawling : SGOL
start crawling : OUNZ
start crawling : UGLD
start crawling : DGL
start crawling : UGL
start crawling : DGP
start crawling : GLL

In [94]:
all_data_week = all_data_week.dropna()

In [95]:
all_data_month = all_data_month.dropna()

週報酬率=(本週收盤價-上週收盤價)/上週收盤價  
月報酬率=(本月收盤價-上月收盤價)/上月收盤價

In [96]:
def  calculate_ra_1(etf): # calculate one ra
    today_etf=etf[-1]
    return [(today_etf-etf[i])/etf[i] for i in range(0,len(etf)-1)]

In [97]:
def calculate_ra_all(date,etfs,all_data):
    dict={}
    for etf in etfs:
        dict[etf]=calculate_ra_1(all_data[etf])
    
    df = pd.DataFrame.from_dict(dict)
    return df

In [98]:
ra_wk = calculate_ra_all(all_data_week.index,ETFLISTwk,all_data_week)
ra_mo = calculate_ra_all(all_data_month.index,ETFLISTmo,all_data_month)

In [99]:
ra_wk

,DBO,DGL,DGLD,DGP,DGZ,DTO,DZZ,GLD,GLL,IAU,...,OUNZ,SCO,SGOL,SZO,UBG,UCO,UGL,UGLD,USL,USO
0,0.194522,0.154970,-0.502109,0.259647,-0.156725,-0.680749,-0.332582,0.196826,-0.364154,0.204301,...,0.197814,-0.763544,0.197797,-0.392694,0.192634,-0.098884,0.232762,0.254545,0.350176,0.167273
1,0.336270,0.111362,-0.435775,0.161668,-0.120195,-0.741764,-0.272727,0.149035,-0.307345,0.155722,...,0.149726,-0.808531,0.150868,-0.460525,0.131772,0.132265,0.135378,0.112635,0.468371,0.310204
2,0.484948,0.127006,-0.456304,0.197589,-0.132892,-0.774431,-0.296912,0.166699,-0.327459,0.172217,...,0.164603,-0.842661,0.168112,-0.504184,0.156016,0.414268,0.172050,0.165952,0.636752,0.460751
3,0.407607,0.115077,-0.440057,0.175412,-0.125531,-0.759573,-0.281553,0.156476,-0.313159,0.163362,...,0.154995,-0.818328,0.157855,-0.489317,0.146601,0.288483,0.153241,0.138670,0.542282,0.385113
4,0.351303,0.097348,-0.408789,0.137382,-0.109327,-0.745616,-0.253468,0.135390,-0.289436,0.142857,...,0.135225,-0.798469,0.136518,-0.471381,0.131772,0.194503,0.110606,0.077622,0.447103,0.330570
5,0.447179,0.043296,-0.311963,0.031603,-0.060586,-0.777739,-0.167370,0.081108,-0.210526,0.087379,...,0.079870,-0.821146,0.082147,-0.506513,0.131772,0.421384,0.008808,-0.069819,0.506885,0.441077
6,0.501448,-0.009617,-0.176922,-0.064292,0.000000,-0.795412,-0.046699,0.025938,-0.116377,0.031826,...,0.023639,-0.833245,0.026414,-0.515613,0.004966,0.632948,-0.089667,-0.200173,0.577213,0.541417
7,0.509836,-0.003702,-0.191161,-0.081961,-0.008935,-0.788264,-0.066246,0.032744,-0.126749,0.038786,...,0.032023,-0.824034,0.034207,-0.515739,0.020807,0.582633,-0.078451,-0.181770,0.568601,0.501754
8,0.493153,0.001783,-0.195982,-0.057732,-0.013680,-0.777582,-0.075000,0.036888,-0.131588,0.043184,...,0.036266,-0.810427,0.037843,-0.501878,0.027937,0.504660,-0.064574,-0.164770,0.532000,0.462415
9,0.405777,-0.029074,-0.118536,-0.103922,0.014065,-0.728777,-0.006711,0.007383,-0.075330,0.012325,...,0.005666,-0.771296,0.007395,-0.469265,-0.011901,0.256952,-0.120892,-0.237129,0.422030,0.330570


In [100]:
ra_mo

,DBO,DGL,DGLD,DGP,DGZ,DTO,DZZ,GLD,GLL,IAU,...,OUNZ,SCO,SGOL,SZO,UBG,UCO,UGL,UGLD,USL,USO
0,0.386864,0.095956,-0.406911,0.131409,-0.108091,-0.758617,-0.252207,0.133240,-0.288278,0.140074,...,0.133784,-0.811607,0.134501,-0.471381,0.124432,0.282241,0.108485,0.075991,0.489924,0.377202
1,0.503375,-0.013072,-0.159957,-0.074511,0.002778,-0.779069,-0.043548,0.021578,-0.104950,0.025021,...,0.021427,-0.813239,0.022645,-0.489598,0.021270,0.533502,-0.095002,-0.200693,0.540365,0.476667
2,0.427916,-0.007437,-0.164845,-0.091890,-0.002762,-0.740801,-0.097413,0.030262,-0.111526,0.034512,...,0.029869,-0.774400,0.030565,-0.444655,0.008780,0.361392,-0.078357,-0.174240,0.450644,0.370103
3,0.290105,-0.054835,-0.019052,-0.189372,0.050182,-0.626627,0.051418,-0.019814,-0.007667,-0.015224,...,-0.019860,-0.677722,-0.018014,-0.338048,0.002805,0.028838,-0.158306,-0.283786,0.290076,0.176106
4,0.224603,0.006557,-0.181865,-0.060744,-0.010959,-0.617473,-0.066142,0.044287,-0.119164,0.048635,...,0.041805,-0.638522,0.045278,-0.304744,0.030099,-0.058960,-0.040147,-0.127410,0.228453,0.119629
5,0.211234,-0.073883,0.082990,-0.203295,0.085714,-0.593689,0.138196,-0.041670,0.053857,-0.036834,...,-0.041216,-0.638079,-0.040403,-0.312469,-0.062919,-0.007365,-0.187653,-0.318672,0.222739,0.148660
6,0.376539,-0.093825,0.154240,-0.219169,0.102290,-0.689001,0.176587,-0.060319,0.106661,-0.056068,...,-0.062221,-0.731966,-0.059798,-0.442095,-0.056598,0.413753,-0.217373,-0.352140,0.399172,0.361680
7,0.316122,-0.054165,0.043852,-0.156586,0.056328,-0.659002,0.098148,-0.028691,0.034933,-0.024603,...,-0.028935,-0.688528,-0.028500,-0.375252,-0.062919,0.280887,-0.161778,6.235110,0.335968,0.282819
8,0.250835,-0.061271,0.074637,-0.153760,0.077612,-0.611611,0.116761,-0.035339,0.051004,-0.031521,...,-0.035632,-0.634858,-0.034639,-0.334727,-0.067536,0.168593,-0.168825,-0.287654,0.269995,0.215919
9,0.294622,-0.031950,-0.011890,-0.096582,0.041847,-0.628682,0.053286,-0.006069,-0.006331,-0.002435,...,-0.005924,-0.653855,-0.004825,-0.344571,-0.014400,0.260915,-0.114929,6.870418,0.308628,0.262108


In [169]:
#測試用資料
data = [0.0089,0.0012,-0.002,0.01,-0.0002,0.02,0.03,0.01,-0.003,0.01,0.0102,-0.01]

In [170]:
import numpy as np
from numpy import mean
from numpy import var
from scipy.stats import kurtosis
from scipy.stats import skew
import math
import cmath

def sharpeASKSR(R, rf):
    x = np.asarray(R)
    u = mean(x)/x.size    #期望值
    v = var(x, ddof=1)    #變異數
    SD = math.sqrt(v)     #標準差
    S = skew(x)           #偏度
    K = kurtosis(x)       #峰度

    c1 = 3*K-4*pow(S,2)-9
    c2 = 3*K-5*pow(S,2)-9
    a = ( 3*(cmath.sqrt(c1) ) / ( v*c2 ) )
    b = ( 3*S ) / ( SD*c2 )
    n = u - ( (3*S*SD) / c1 )
    d = 3*SD * ( (cmath.sqrt(c2)) / (c1) ) 
    phi = cmath.sqrt(pow(a,2)*pow(b,2))
    a_ = b + ( a*(n-rf) / cmath.sqrt(pow(d,2)+pow(n-rf,2))) 
    ASKSR = cmath.sqrt(2*(a_*(n-rf)-d*(phi-cmath.sqrt(pow(a,2)-pow(b-a_,2)))))
    
    return ASKSR
    

In [171]:
sharpeASKSR(data,0)

44.82677858843439j

In [130]:
def OmegaSharpeRatio(R, L):
    L=L/100
    n = len(R)
    upside = 0.0
    downside = 0.0
    
    for i in range(n):
        if R[i] < L:
            downside = downside + (L-R[i])
        else:
            upside = upside + (R[i] - L)
            
    if downside != 0:
        OmegaRatio = (upside / downside)-1
    else:
        OmegaRatio = "undefined"
        
    return OmegaRatio

In [155]:
def Qfunction(datalist):
    Q = 0.0
    for r in datalist:
        ag = -1/r
        Q+=math.exp( ag )
    return Q

In [ ]:
for S in ETFLISTmo:
    A = Qfunction(ra_mo[S])
    print(A)

In [151]:
def risk_sort(listR,dataR):
    risk = []
    for etf in listR:
        risk.append(OmegaSharpeRatio(dataR[etf], 2.24))
    dict={}
    dict['ETF_Symbol'] = listR
    dict['riskvalue'] = risk
    df = pd.DataFrame.from_dict(dict)
    df = df.sort_values(by="riskvalue",ascending=False)
    df = df.reset_index(drop=True)
    return df

In [152]:
print('----------OmegaSharpeRatio(week)----------')
riskwk=risk_sort(ETFLISTwk,ra_wk)
riskwk.head()

----------OmegaSharpeRatio(week)----------


,ETF_Symbol,riskvalue
0,USL,8.347613
1,USO,4.883116
2,DBO,4.356930
3,OLEM,4.280536
4,OLO,4.077740
5,UCO,0.909500
6,DZZ,-0.091024
7,DGZ,-0.193011
8,IAU,-0.256755
9,SGOL,-0.342875


In [153]:
print('----------OmegaSharpeRatio(month)----------')
riskmo=risk_sort(ETFLISTmo,ra_mo)
riskmo

----------OmegaSharpeRatio(month)----------


,ETF_Symbol,riskvalue
0,USL,13.131988
1,USO,8.582110
2,OLEM,6.476022
3,DBO,6.332680
4,UGLD,5.377849
5,OLO,3.859192
6,UCO,2.519754
7,DZZ,0.053840
8,DGZ,0.009224
9,IAU,-0.420459
